# XYZ's IT User Access Review Step 4: Generate Analytical Reports & PowerBI datasets

###### Author : Sateesh Babu
###### Version: V3
###### Created: 2021-12-23
###### Updated: 2021-12-30

# Input

In [1]:
# Date when you want to generate the reports for give xyz IT AD snapshot
run_date = '20211230'

In [2]:
# Source data file path
# 1.HR data; 2.xyz IT AD ; 3.Control file: xyz IT AD departments list; 4.Control file: xyz IT AD Group list

source_filepath1 = r"C:\Users\SATEESHB\OneDrive - Metrolinx\documents\zMyWork\05_PowerBI_PythonCode\00_Py_UserAccess\Source_Data\IT_AD"
source_filepath2 = r"C:\Users\SATEESHB\OneDrive - Metrolinx\documents\zMyWork\05_PowerBI_PythonCode\00_Py_UserAccess\Results_Data\Step1_Results_HR"
source_filepath3 = r"C:\Users\SATEESHB\OneDrive - Metrolinx\documents\zMyWork\05_PowerBI_PythonCode\00_Py_UserAccess\Source_Data\Control_Files"
source_filepath4 = r"C:\Users\SATEESHB\OneDrive - Metrolinx\documents\zMyWork\05_PowerBI_PythonCode\00_Py_UserAccess\Results_Data\Step2_Results_AD_Master"

source_ad_file              = source_filepath1 + f"\IT_AD_XYZ_Users_{run_date}.xlsx"
source_hr_file              = source_filepath2 + f"\df_final_hr_dataset_{run_date}.xlsx" 
source_ad_ctrl_dept_file  = source_filepath3 + f"\AD_Ctrl_Departments.xlsx"
source_ad_ctrl_grps_file  = source_filepath3 + f"\AD_Ctrl_XYZ_Groups.xlsx"
source_ad_userassign_file   = source_filepath4 + f"\output_ad_user_group_assignments_{run_date}.xlsx"

In [3]:
# results/outcome of this program

results_1st     = r"C:\Users\SATEESHB\OneDrive - Metrolinx\documents\zMyWork\05_PowerBI_PythonCode\00_Py_UserAccess\Results_Data\Step4_Results_Ouput\1_Line_of_Defense"
results_2nd     = r"C:\Users\SATEESHB\OneDrive - Metrolinx\documents\zMyWork\05_PowerBI_PythonCode\00_Py_UserAccess\Results_Data\Step4_Results_Ouput\2_Line_of_Defense"
results_3rd     = r"C:\Users\SATEESHB\OneDrive - Metrolinx\documents\zMyWork\05_PowerBI_PythonCode\00_Py_UserAccess\Results_Data\Step4_Results_Ouput\3_Line_of_Defense"

## Code

### Packages

In [4]:
import numpy as np
import pandas as pd
#import pandasql as ps

from datetime import datetime
from datetime import date
from datetime import time
from dateutil import relativedelta
from datetime import timedelta
import time

import warnings
warnings.filterwarnings('ignore')

### Functions

In [5]:
# Stats
def df_stats(df,columns):
    '''
    Stats of each column:
    1.Count
    2.Unique Values count
    3.Missing Values count 
    4.% of Missing Values wrt total instances of the dataset
    5.% of Highest column value count
    6.Data Type
    
    '''
    
    stats = []
    for col in columns:
        stats.append((col, df[col].count(), df[col].nunique(), df[col].isnull().sum(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    
    stats_cols = ['feature', 'record_count', 'unique_values','missing_records', 'percent_missing_records','percent_highest_columnvalue', 'data_type']
    df_stats = pd.DataFrame(stats, columns=stats_cols)
    df_stats['cat_or_num'] = df_stats['data_type'].apply(lambda x: 'categorical' if x == 'object' else 'numerical')
    return(df_stats)

### Get Data

In [6]:
df_hr_dataset = pd.read_excel(source_hr_file)
df_ad_dataset = pd.read_excel(source_ad_file)
df_ad_userassign_dataset = pd.read_excel(source_ad_userassign_file)
df_ctrl_dept = pd.read_excel(source_ad_ctrl_dept_file)
df_ctrl_grps = pd.read_excel(source_ad_ctrl_grps_file)

In [7]:
print(f'Shape of HR data: {df_hr_dataset.shape}')
print(f'Shape of AD data: {df_ad_dataset.shape}')
print(f'Shape of AD user assignments data: {df_ad_userassign_dataset.shape}')
print(f'Shape of AD ctrl groups data: {df_ctrl_grps.shape}')
print(f'Shape of AD ctrl dept data: {df_ctrl_dept.shape}')

Shape of HR data: (42, 16)
Shape of AD data: (50, 19)
Shape of AD user assignments data: (461, 3)
Shape of AD ctrl groups data: (23, 5)
Shape of AD ctrl dept data: (10, 4)


In [8]:
cols = df_ad_dataset.columns
#df_stats(df_ad_dataset,cols)

### Prepare Data

In [9]:
# get data without user groups
cols1 = list(set(cols) - set(['Group']))
df_ad1 = df_ad_dataset[cols1]

df_ad1.head(3)

,Lastname,OUpath,SamAccountName,DisplayName,EmailAddress,CreateTimeStamp,WhenChanged,WhenCreated,AdminCount,UserAccountControl,Description,Firstname,Company,AccountExpiry,Passwordset,Enabled,LastLogonDate,Department
0,Emp1,Test path,JOHNEMP1x,John Emp1,John.Emp12@xyz.com,2021-05-18 03:20:58,2021-12-01 01:00:17,2021-05-18 03:20:58,NaN,512,Request#123,John,NaN,NaN,2021-09-17 01:02:31,True,2021-06-16 01:00:09,XYZ
1,Emp2,Test path,KINEMP2x,Kin Emp2,Kin.Emp2@xyz.com,2021-05-18 03:20:58,2021-12-01 01:00:17,2021-05-18 03:20:58,NaN,514,Request#122,Kin,NaN,NaN,2021-09-17 01:02:31,False,2021-06-16 01:00:09,xyz
2,Emp3,Test path,JUNEMP3x,Jun Emp3,Jun.Emp3@xyz.com,2021-09-18 12:31:40,2021-09-20 13:25:07,2021-09-18 12:31:40,NaN,512,Request#121,Jun,NaN,NaN,2021-09-17 12:31:40,True,2021-09-20 13:07:35,XYZ_Dept1


In [10]:
# get data with user groups
df_ad2 = df_ad_dataset[['SamAccountName','Group']]
df_ad2.head(3)

,SamAccountName,Group
0,JOHNEMP1x,grp1;grp2;grp3
1,KINEMP2x,grp1;grp2;grp3;grp4
2,JUNEMP3x,grp1;grp2;grp3;grp4;grp5;grp6;grp7;grp8;grp9;g...


In [11]:
df_ad3 = df_ad_userassign_dataset
print(df_ad3.head(5))

   Unnamed: 0 SamAccountName Group
0           0      JOHNEMP1x  grp1
1           1      JOHNEMP1x  grp2
2           2      JOHNEMP1x  grp3
3           0       KINEMP2x  grp1
4           1       KINEMP2x  grp2


### Joins

In [12]:
# Join 1: Identify xyz AD Groups in each user group provision
df_m = pd.merge(df_ad3,df_ctrl_grps[['Unique_AD_Groups','xyz_Grp_Tags','Privileged_Access']], left_on = 'Group', right_on = 'Unique_AD_Groups',how ='left')
Q = (df_m["xyz_Grp_Tags"]=="X")
df_xyz_usersgrps = df_m[Q][["SamAccountName","Group","xyz_Grp_Tags","Privileged_Access"]]
df_xyz_usersgrps.head(3)

,SamAccountName,Group,xyz_Grp_Tags,Privileged_Access
0,JOHNEMP1x,grp1,X,NaN
2,JOHNEMP1x,grp3,X,NaN
3,KINEMP2x,grp1,X,NaN


In [13]:
# Join 2: Get additional details of the AD user
df_ad_xyz1 = pd.merge(df_ad1,df_xyz_usersgrps, on = 'SamAccountName', how ='left')

In [14]:
# Join 3: Identify xyz Groups in each user group provision
df_ad_xyz2 = pd.merge(df_ad_xyz1,df_ctrl_dept[['Unique_AD_Departments','xyz_Dept_Tags']], left_on = 'Department', right_on = 'Unique_AD_Departments',how ='left')

In [15]:
# HR work email is not null 
q = df_hr_dataset["Work_Email"].notnull()
df_hr_dataset2 = df_hr_dataset[q]
df_hr_dataset2.shape

(42, 16)

In [16]:
# Join 4: Get HR details for each user

df_ad_xyz21 = df_ad_xyz2
df_hr_dataset21 = df_hr_dataset2

df_ad_xyz21.columns = df_ad_xyz2.columns.map(lambda x: 'AD_'+ str(x))
df_hr_dataset21.columns = df_hr_dataset2.columns.map(lambda x: 'HR_'+ str(x))

df_ad_xyz21['AD_EmailAddress'] = df_ad_xyz21['AD_EmailAddress'].str.strip().str.lower()
df_hr_dataset21['HR_Work_Email'] = df_hr_dataset21['HR_Work_Email'].str.strip().str.lower()




# spilt AD datasets
q = df_ad_xyz21["AD_EmailAddress"].notnull()
df_ad_xyz22 = df_ad_xyz21[q]

q = df_ad_xyz21["AD_EmailAddress"].isnull()
df_ad_xyz23 = df_ad_xyz21[q]


# join with HR data 
df_ad_xyz3 = pd.merge(df_ad_xyz22,df_hr_dataset21, left_on = 'AD_EmailAddress', right_on = 'HR_Work_Email', how ='left', indicator=True)

# concate blank + join 
df_ad_xyz4 = pd.concat([df_ad_xyz3,df_ad_xyz23])

col_remove = ['_merge','HR_Rehire_Flag', 'HR_Person_User_Name','HR_Start_Date','HR_Employee Assignment Type','HR_Position','HR_Unnamed: 0']
col = df_ad_xyz4.columns
re_col = [elem for elem in col if elem not in col_remove]
df_ad_xyz = df_ad_xyz4[re_col]

print(df_ad_xyz.shape)
df_ad_xyz.head(3)

(288, 33)


,AD_AccountExpiry,AD_AdminCount,AD_Company,AD_CreateTimeStamp,AD_Department,AD_Description,AD_DisplayName,AD_EmailAddress,AD_Enabled,AD_Firstname,...,HR_Department_Name,HR_Division,HR_Employee_No,HR_First_Name,HR_Last_Name,HR_Manage_Email,HR_Record_Type,HR_Status,HR_Termination_Date,HR_Work_Email
0,NaN,NaN,NaN,2021-05-18 03:20:58,XYZ,Request#123,John Emp1,john.emp12@xyz.com,True,John,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,NaN,NaN,NaN,2021-05-18 03:20:58,XYZ,Request#123,John Emp1,john.emp12@xyz.com,True,John,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,NaN,NaN,NaN,2021-05-18 03:20:58,xyz,Request#122,Kin Emp2,kin.emp2@xyz.com,False,Kin,...,Salesforce,Consulting,1002.0,Kin,Emp2,Sat.Emp5@xyz.com,XYZ Employee,Active,NaT,kin.emp2@xyz.com


### Transformations

##### AD Account Status

In [17]:
# Transform AD Account Status using Enabled

df_ad_xyz['AD_Enabled']      = df_ad_xyz['AD_Enabled'].astype(str)
df_ad_xyz['zAD_User_Status'] = df_ad_xyz['AD_Enabled'].apply(lambda x: 'Active' if x == 'True' else 'Inactive')

In [18]:
q = (df_ad_xyz['AD_SamAccountName']=='JOHNEMP1x')
col = ['AD_SamAccountName', 'zAD_User_Status', 'AD_Enabled']

df_ad_xyz[col][q]

,AD_SamAccountName,zAD_User_Status,AD_Enabled
0,JOHNEMP1x,Active,True
1,JOHNEMP1x,Active,True


##### Teriminated Users Access Check

In [19]:
# Auditcheck: Check the active AD access of terminated employees. 

df_ad_xyz['zTerminated_User_ActiveAD'] = df_ad_xyz[['zAD_User_Status','HR_Status']].apply(lambda x: "AuditCheck" if x['zAD_User_Status'] == "Active" and x['HR_Status'] == "Departed" else '', axis=1)

In [20]:
col = ['AD_SamAccountName', 'zAD_User_Status', 'AD_Enabled','HR_Status', "HR_Termination_Date",'zTerminated_User_ActiveAD']
Q = (df_ad_xyz['zTerminated_User_ActiveAD'] == "AuditCheck")
df_ad_xyz[Q][col].head(5)

,AD_SamAccountName,zAD_User_Status,AD_Enabled,HR_Status,HR_Termination_Date,zTerminated_User_ActiveAD
154,FAYEMP27x,Active,True,Departed,2019-01-01,AuditCheck
155,FAYEMP27x,Active,True,Departed,2019-01-01,AuditCheck
156,FAYEMP27x,Active,True,Departed,2019-01-01,AuditCheck
157,FAYEMP27x,Active,True,Departed,2019-01-01,AuditCheck
158,FAYEMP27x,Active,True,Departed,2019-01-01,AuditCheck


##### Aging

In [21]:
#  Age1: Days since last login date.
#  Age2: Days since last password reset.
#  Age3: Days since last update (when changed).
#  Age4: Days since creation (when created).

def age_calc(y):
    x = date.today()
    y = y.date()
    diff = (x-y)
    return diff.days

df_ad_xyz['zDays_SinceLastLogonDate'] = df_ad_xyz[['AD_LastLogonDate']].apply(lambda x:age_calc(*x), axis=1)
df_ad_xyz['zDays_SincePswdReset']     = df_ad_xyz[['AD_Passwordset']].apply(lambda x:age_calc(*x), axis=1)
df_ad_xyz['zDays_SinceWhenChanged']   = df_ad_xyz[['AD_WhenChanged']].apply(lambda x:age_calc(*x), axis=1)
df_ad_xyz['zDays_SinceWhenCreated']   = df_ad_xyz[['AD_WhenCreated']].apply(lambda x:age_calc(*x), axis=1)

### Filter

##### Mismatches

In [22]:
# Mismatches with HR email
q=(df_ad_xyz['HR_Work_Email'].isnull()) | (df_ad_xyz['HR_Work_Email']==u'')
df_ad_xyz_mismatches = df_ad_xyz[q]
df_ad_xyz_mismatches["zMatched"] = "MisMatch"

##### HR Matches

In [23]:
# HR matches with HR email
q=(df_ad_xyz['HR_Work_Email'].notnull())
df_ad_xyz_hrmatches = df_ad_xyz[q]
df_ad_xyz_hrmatches["zMatched"] = "HR_Match"

In [24]:
df_ad_xyz1 = pd.concat([df_ad_xyz_hrmatches,df_ad_xyz_mismatches])
df_ad_xyz1["Reporting_Date"] = run_date

## Output

In [25]:
group_cols = ['AD_SamAccountName','AD_DisplayName', 'AD_Group', 'AD_xyz_Grp_Tags', 'AD_Privileged_Access',"zMatched","Reporting_Date"]
user_cols = ['AD_SamAccountName','AD_DisplayName','AD_AccountExpiry', 'AD_Company', 'AD_CreateTimeStamp', 'AD_Department', 'AD_Description', 'AD_EmailAddress',
       'AD_Enabled', 'AD_Firstname', 'AD_LastLogonDate', 'AD_Lastname', 'AD_Passwordset', 'AD_xyz_Dept_Tags', 'AD_WhenChanged', 'AD_WhenCreated', 'HR_Department_Name', 'HR_Division', 'HR_Employee_No', 'HR_First_Name', 'HR_Last_Name', 'HR_Manage_Email', 'HR_Record_Type',
       'HR_Status', 'HR_Termination_Date', 'HR_Work_Email', 'zAD_User_Status', 'zTerminated_User_ActiveAD', 'zDays_SinceLastLogonDate', 'zDays_SincePswdReset', 'zDays_SinceWhenChanged', 'zDays_SinceWhenCreated',"zMatched","Reporting_Date"]

##### Split Users and their User assignments(Groups)

In [26]:
df_ad_xyz1_grps = df_ad_xyz1[group_cols]
df_ad_xyz1_users = df_ad_xyz1[user_cols].drop_duplicates(subset=['AD_SamAccountName'],keep='last')

##### 1. Power BI inputs files (1st Line of Defense's ASK)

In [27]:
# Refined datasets for Power BI visualizations which is used for daily monitoring activity.
with pd.ExcelWriter(results_1st+f"\output_ad_xyz_{run_date}.xlsx") as writer:
    df_ad_xyz1_users.to_excel(writer, sheet_name="ad_xyz_users")
    df_ad_xyz1_grps.to_excel(writer, sheet_name="ad_xyz_grps")

##### 2. Manager Review Reports (2nd line of Defense's ASK)

In [28]:
# Need a Manager's report with all hi/her reportees and their group assignments.

hr_managers = df_ad_xyz_hrmatches['HR_Manage_Email'].unique().tolist()

for i in hr_managers:
    q = (df_ad_xyz_hrmatches['HR_Manage_Email']== i)
    df = df_ad_xyz_hrmatches[q]
    cols = df.columns
    cols1 = list(set(cols) - set(['AD_Group','AD_AdminCount',"AD_UserAccountControl","AD_OUpath"]))
    df_user=df[cols1].drop_duplicates(['AD_SamAccountName'], keep ='last')
    df_user_grps=df[["AD_SamAccountName","AD_DisplayName", "AD_Group","AD_xyz_Grp_Tags", "AD_Privileged_Access","zMatched"]] 
    
    with pd.ExcelWriter(results_2nd+f"\output_ad_xyz_ManagerReview_{run_date}_{i}.xlsx") as writer:
        #df.to_excel(writer, sheet_name=i[:10])
        df_user.to_excel(writer, sheet_name="AD_Users")
        df_user_grps.to_excel(writer, sheet_name="Their_Groups")

##### 3. Audit Review Reports (3rd line of Defense's ASK)

In [29]:
# Terminated User with active AD Access + all Mismatches including contractors

q = (df_ad_xyz_hrmatches['zTerminated_User_ActiveAD'] == "AuditCheck")
df_hr = df_ad_xyz_hrmatches[q]
df = pd.concat([df_hr,df_ad_xyz_mismatches])

cols = df.columns
cols1 = list(set(cols) - set(['AD_Group','AD_AdminCount',"AD_UserAccountControl","AD_OUpath"]))
df_user=df[cols1].drop_duplicates(['AD_SamAccountName'], keep ='last')
df_user_grps=df[["AD_SamAccountName","AD_DisplayName", "AD_Group","AD_xyz_Grp_Tags", "AD_Privileged_Access","zMatched"]] 
    
with pd.ExcelWriter(results_3rd+f"\output_ad_xyz_AuditReview_{run_date}_{i}.xlsx") as writer:
    df_user.to_excel(writer, sheet_name="AD_Users")
    df_user_grps.to_excel(writer, sheet_name="Their_Groups")

In [30]:
df_ad_xyz1_grps['AD_Group'].unique()

array(['grp1', 'grp3', 'grp5', 'grp6', 'grp8', 'grp10', 'grp11', 'grp12',
       'grp15', nan], dtype=object)